# 🏗 Groundwork 🪚

In [1]:
from os import system, remove
from os.path import isfile
from glob import glob
from pandas import ExcelWriter, ExcelFile, read_excel
from openpyxl import load_workbook

xlsx = 'unite.xlsx'
chief = 'chief'
cell = 'C10'

system('pip show pandas')

Name: pandas
Version: 1.4.4
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /Users/v.syroiezhin/opt/anaconda3/lib/python3.9/site-packages
Requires: numpy, python-dateutil, pytz
Required-by: datashader, holoviews, hvplot, seaborn, statsmodels, xarray


0

# 📋 Sketch

#### Delete the file with the reserved name, if it exists, and create a file to merge the contents

In [2]:
def capacity(typefile)->list: return sorted([ file for file in glob(f'*.{typefile}') if '~$' not in file ])
def review(message:str, typefile:str): print(message, '\n', capacity(typefile), '\n')

review('Before launch it was like this :','xlsx')

if isfile(xlsx): 
    remove(xlsx)
    review('Deleted the file named unite.xlsx :','xlsx')
fxls = capacity('xlsx')

ExcelWriter(xlsx).save()
review('Created a file named unite.xlsx :','xlsx')

Before launch it was like this : 
 ['ID.xlsx', 'packet.xlsx', 'payment.xlsx', 'unite.xlsx'] 

Deleted the file named unite.xlsx : 
 ['ID.xlsx', 'packet.xlsx', 'payment.xlsx'] 

Created a file named unite.xlsx : 
 ['ID.xlsx', 'packet.xlsx', 'payment.xlsx', 'unite.xlsx'] 



### Looping through files and pages of files to save their contents to a new file

In [3]:
def fill(file:str, namesheet:str, content:list, columns:list=None, mode:str='a', index=False, header=True):
    with ExcelWriter(file, mode=mode, engine="openpyxl", if_sheet_exists=(None if mode=='w' else 'overlay')) as writer:  
        content.to_excel(writer, sheet_name=namesheet, index=index, header=header, columns=columns)
        
def unite(fxls:list):
    list_sheet=[]
    for fxl in fxls:
        ef = ExcelFile(fxl)
        print('\n', fxl, ":", end=' ')
        for sheet in ef.sheet_names:
            list_sheet.append(sheet) if sheet not in list_sheet else list_sheet.append(sheet+'_'+fxl.replace('.xlsx',''))
            fill(content=read_excel(ef, sheet_name=sheet), file=xlsx, namesheet=list_sheet[-1], mode='a', header=True)
            print(list_sheet[-1], end=' ')

unite(fxls)


 ID.xlsx : ID 
 packet.xlsx : packet 
 payment.xlsx : payment 

#### When we create a file, a page with a known name is also created, we can rename this page

In [4]:
def rename(file:str, sheet:str, oldnamesheet:str='Sheet1'):
    fxl = load_workbook(file)
    fxl[oldnamesheet].title = sheet
    fxl.save(file)

def check(file:str, message:str='\nResult of checking:'):
    fxl = load_workbook(file)
    sheets = fxl.sheetnames
    print(message, sheets)

check(xlsx, 'Before:')
rename(xlsx, chief)
check(xlsx, 'After:')

Before: ['Sheet1', 'ID', 'packet', 'payment']
After: ['chief', 'ID', 'packet', 'payment']


### Write two lines of data to a file, disabling the names of rows and columns

In [5]:
from pandas import DataFrame

df = DataFrame(
    [['1A', '1B','1C'], 
     ['2A', '2B','2C'], 
     ['3A', '3B','3C']],
     columns=['A', 'B', 'C'],
       index=['1', '2', '3'] 
    )

fill(file=xlsx, namesheet=chief, content=df, columns=['C','A','B'])

### Read information from a cell

In [6]:
def scan(file:str, namesheet:str, start:str='A', upto:str='A1') -> DataFrame:
    tabl = []
    fxl = load_workbook(file)
    cells = fxl[namesheet][start+'1':upto]
    columns = [ cells[0][c].value for c in range(len(cells[0])) ]
    for cell in cells[1:]: tabl.append([ cell[c].value for c in range(len(cell)) ])
    return DataFrame( tabl, columns=columns, index=list(range(1,int(upto[1:]))) )

df = scan(file=xlsx, namesheet=chief, upto=cell)
print(f'In cells "A1:{cell}" of the "{chief}" page of the file "{xlsx}" we got the contents of \n{df}')

fill(file=xlsx, namesheet=chief, content=df)

In cells "A1:C10" of the "chief" page of the file "unite.xlsx" we got the contents of 
      C     A     B
1    1C    1A    1B
2    2C    2A    2B
3    3C    3A    3B
4  None  None  None
5  None  None  None
6  None  None  None
7  None  None  None
8  None  None  None
9  None  None  None


### To see the result in the file run this...

In [7]:
system(f''' open {xlsx}; 
            sleep 10; 
            killall "Microsoft Excel" || taskkill "Microsoft Excel" ''') # mac || win

0